<a href="https://colab.research.google.com/github/KyleHung7/programming-language/blob/main/Week_12_FinTech_Article_Analysis_using_CKIPtagger.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **利用 CKIPtagger 進行FinTech文本分析**

### **為什麼我們要處理文本？**

#### 在我的這次研究作業中，我選擇深入探討中文自然語言處理 (NLP) 的一個核心應用：文本分析。在當今資訊爆炸的時代，如何從大量的文字資料中萃取出有價值的資訊，並理解其中的模式，是許多領域，包括金融科技 (FinTech)，都非常關注的議題。我將利用台灣語言學計算所開發的 CKIPtagger 工具，對一段關於「金融科技」的中文文本進行斷詞、詞性標註與命名實體識別，最終透過視覺化圖表呈現分析結果。

### **步驟一：Colab 環境建置與 TensorFlow 版本調整**

#### 在 Colab 環境中進行深度學習相關的研究時，常常會遇到套件版本相容性的問題。特別是像 CKIPtagger 這樣基於特定 TensorFlow 版本開發的工具，如果環境中的 TensorFlow 版本過新，就可能導致程式無法正常運行。

#### 因此，我的第一步，也是最重要的一步，就是確保我們使用的 TensorFlow 版本符合 CKIPtagger 的需求。我選擇了 TensorFlow 2.15.0 這個版本，它在引入 Keras 3 之前的版本中算是較新且對 Python 3.11 支援較好的

In [5]:
# -*- coding: utf-8 -*-

# 確保程式碼能夠正確處理中文字元。

TARGET_TF_VERSION = "2.15.0"

print(f"--- 步驟 1: 檢查及設定 TensorFlow 版本為 {TARGET_TF_VERSION} ---")

try:
    # 嘗試匯入 TensorFlow
    import tensorflow as tf
    print(f"目前 Colab 環境中的 TensorFlow 版本是: {tf.__version__}")

    if tf.__version__ != TARGET_TF_VERSION:
        print(f"檢測到 TensorFlow 版本不是 {TARGET_TF_VERSION}。")
        print("正在卸載當前版本並安裝指定版本...")
        # 使用 !pip 執行 shell 命令
        !pip uninstall tensorflow -y # 移除當前版本的 TensorFlow
        !pip install tensorflow=={TARGET_TF_VERSION} # 安裝指定版本

        print(f"\n--- TensorFlow {TARGET_TF_VERSION} 安裝指令已發出 ---")
        print("\n🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨")
        print("🚨🚨🚨 最重要的步驟：請務必手動點擊「執行階段」->「重新啟動執行階段」或按 Ctrl+M . 🚨🚨🚨")
        print("🚨🚨🚨 重新啟動後，請再從頭執行一次這個儲存格。 🚨🚨🚨")
        print("🚨🚨🚨 這是因為 TensorFlow 核心套件變更，需要重新載入環境才能生效。 🚨🚨🚨")
        print("🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨")

        # 強制退出，提醒使用者必須重新啟動。這個步驟是必須的。
        import os
        os._exit(00) # 使用 exit code 00 表示正常終止，但需要重啟

    else:
        print(f"TensorFlow 版本已是 {TARGET_TF_VERSION}，符合要求，無需額外操作。")

except ImportError:
    # 如果 TensorFlow 尚未安裝，直接安裝指定版本
    print(f"TensorFlow 尚未安裝，直接安裝 {TARGET_TF_VERSION} 版本。")
    !pip install tensorflow=={TARGET_TF_VERSION}

    print(f"\n--- TensorFlow {TARGET_TF_VERSION} 安裝指令已發出 ---")
    print("\n🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨")
    print("🚨🚨🚨 最重要的步驟：請務必手動點擊「執行階段」->「重新啟動執行階段」或按 Ctrl+M . 🚨🚨🚨")
    print("🚨🚨🚨 重新啟動後，請再從頭執行一次這個儲存格。 🚨🚨🚨")
    print("🚨🚨🚨 這是因為 TensorFlow 核心套件變更，需要重新載入環境才能生效。 🚨🚨🚨")
    print("🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨🚨")




--- 步驟 1/2: 檢查及設定 TensorFlow 版本為 2.15.0 ---
目前 Colab 環境中的 TensorFlow 版本是: 2.15.0
TensorFlow 版本已是 2.15.0，符合要求，無需額外操作。


### **步驟二：確認 TensorFlow 版本與安裝核心工具套件**

### 在經歷了上一步的環境重新啟動之後，我們需要做的第一件事就是再次確認 TensorFlow 的版本是否已經正確地被設定為我們想要的 2.15.0。這是確保 ckiptagger 能夠正常運作的基礎。

### 接著，我就會安裝本次研究需要用到的兩個主要 Python 套件：

1.**ckiptagger**: 這就是我們的中文自然語言處理神器，由台灣語言學計算所開發，提供高品質的斷詞、詞性標註和命名實體識別功能。

2.**plotly**: 我選擇 plotly.express 來進行數據視覺化。相較於其他繪圖庫，Plotly 可以生成互動式圖表，讓我的分析結果更具表現力，也更容易探索數據。

In [6]:
# --- 如果你已經重新啟動執行階段，且上面的步驟沒有執行 os._exit(00)，則會繼續執行這裡 ---

print(f"\n--- 步驟 2: 安裝 ckiptagger、Plotly 及其他套件 ---")
# 確認 TensorFlow 版本是否正確（這行只有在重啟後且版本正確時執行）
import tensorflow as tf
print(f"確認：重新啟動後當前 TensorFlow 版本為: {tf.__version__}")
if tf.__version__ != TARGET_TF_VERSION:
     print(f"警告：TensorFlow 版本不是預期的 {TARGET_TF_VERSION}。`ckiptagger` 可能無法正常工作。")
     print("請再次確認是否已手動重新啟動執行階段，並從頭運行此儲存格。")
else:
     print("TensorFlow 版本確認無誤。")


# 安裝 ckiptagger 和 plotly 套件 (這部分只在 TF 版本正確後執行)
print("正在安裝 ckiptagger、tqdm 和 plotly 套件，這可能需要一些時間，請稍候...")
!pip install ckiptagger tqdm plotly==5.22.0
print("--- 核心套件安裝完成！---")

# 匯入你接下來會用到的函式庫
print("\n--- 匯入函式庫 ---")
import os
from ckiptagger import data_utils, WS, POS, NER
from collections import Counter
import plotly.express as px
import pandas as pd
print("--- 函式庫匯入完成！現在可以開始使用 ckiptagger 等功能了 ---")


--- 步驟 2/2: 安裝 ckiptagger、Plotly 及其他套件 ---
確認：重新啟動後當前 TensorFlow 版本為: 2.15.0
TensorFlow 版本確認無誤。
正在安裝 ckiptagger、tqdm 和 plotly 套件，這可能需要一些時間，請稍候...
--- 核心套件安裝完成！---

--- 匯入函式庫 ---
--- 函式庫匯入完成！現在可以開始使用 ckiptagger 等功能了 ---


### **步驟三：CKIPtagger 模型資源準備**

#### 就像一個孩子需要學習語言一樣，CKIPtagger 在進行文本分析前，也需要載入預先訓練好的語言模型。這些模型包含了大量的語言知識，讓工具能夠理解詞語的上下文、詞性以及識別特定實體。

#### 這個下載過程通常會持續幾分鐘，因為模型檔案大約 300MB。不過別擔心，Colab 會把這些模型下載到你的執行環境中，所以一旦下載完成，之後在同一個執行階段中就不需要再重複下載了。

In [8]:
# 步驟 3: 下載CKIP模型
# 這裡我讓模型下載到 Colab 的當前工作目錄中。
# 如果模型已經存在，ckiptagger 會自動跳過下載，非常方便。
print("\n--- 步驟 3: 正在下載CKIP模型，這大約需要 300MB 的資料，請耐心等候 ---")
# data_utils.download_data() 會檢查當前目錄是否有 'data' 子目錄並下載模型到其中。
data_utils.download_data(os.getcwd())
print("--- CKIP 模型下載完成！---")

# 步驟 4: 初始化CKIP模型與執行分析前的準備
print("\n--- 步驟 4: 正在載入並初始化CKIP模型，請稍候，這是進行分析前的最後準備 ---")
# 我需要初始化三個核心功能：
# WS (Word Segmentation): 斷詞，將句子切分成有意義的詞語。
# POS (Part-of-Speech Tagging): 詞性標註，判斷每個詞的詞性（例如：名詞、動詞、形容詞）。
# NER (Named Entity Recognition): 命名實體識別，找出文本中的人名、地名、組織、時間等特定實體。
ws = WS("./data") # 指定模型檔案所在的目錄
pos = POS("./data")
ner = NER("./data")
print("--- CKIP 模型初始化完成！準備開始文本分析。---")


--- 步驟 3: 正在下載CKIP模型，這大約需要 300MB 的資料，請耐心等候 ---


Downloading...
From (original): https://drive.google.com/uc?id=1efHsY16pxK0lBD2gYCgCTnv1Swstq771
From (redirected): https://drive.google.com/uc?id=1efHsY16pxK0lBD2gYCgCTnv1Swstq771&confirm=t&uuid=2559108c-380d-4958-9178-937fc3e04724
To: /content/data.zip
100%|██████████| 1.88G/1.88G [00:30<00:00, 62.2MB/s]


--- CKIP 模型下載完成！---

--- 步驟 4: 正在載入並初始化CKIP模型，請稍候，這是進行分析前的最後準備 ---
--- CKIP 模型初始化完成！準備開始文本分析。---


### **步驟四：我的研究文本輸入與前處理**

#### 這是我研究作業中最重要的輸入部分。我將在這裡貼上我選定的中文文本。為了確保分析結果的豐富性和代表性，我的文本長度會盡量保持在 150 字以上。我建議選擇一段完整的文章片段，例如一篇關於金融科技的文章，這樣分析出來的詞彙和實體會更具語義連貫性。

#### 在將文本送入 ckiptagger 處理之前，我會做一個簡單的句子分割。ckiptagger 處理的最小單位是句子，正確的句子分割有助於提升後續斷詞和實體識別的準確度。

In [9]:
# 為了研究作業的完整性，文本長度為150字以上，內容為一篇完整的 FinTech 文章。
text_to_analyze = """
2025年金融科技十大預測
探索人工智慧、永續性和不斷變化的消費者需求如何在 2025 年改變金融科技，從財富管理到 CBDC 和自動化銀行業務
展望 2025 年，在人工智慧、永續性議題和不斷變化的消費者偏好的推動下，金融科技格局將繼續以驚人的速度發展。
從人工智慧財富管理到綠色金融科技解決方案，這十大趨勢正在重塑我們與金錢和金融服務的互動方式。
創新涉及多個領域，包括網路安全、監管技術和分散金融，而 RPA 和嵌入式金融等成熟技術正在達到新的複雜程度。
在這十大趨勢中，我們探討了 2025 年定義金融科技產業的關鍵發展，為金融的未來提供了見解。
10.財富科技
隨著人工智慧和個人化重塑財富管理，財富科技產業預計在 2025 年實現顯著成長。增強型人工智慧驅動的投資組合管理和自動化稅收優化將變得更加突出，而嵌入式金融解決方案將把財富服務整合到日常應用程式中。擁有先進機器人諮詢平台的 Betterment 和以全面財富數據分析而聞名的 Addepar 等公司正在引領這一轉變。千禧世代和 Z 世代投資者對數位優先財富管理解決方案的需求不斷增長，以及開放銀行標準的日益普及，推動了該行業的擴張。

9.永續與綠色金融科技
綠色金融科技將在 2025 年徹底改變永續金融，碳足跡追蹤將無縫整合到銀行應用程式和支付系統中。先進的 ESG 分析和基於區塊鏈的碳信用交易平台將能夠更精確地衡量環境影響。 Aspiration 憑藉其註重氣候的銀行服務，以及利用人工智慧提供 ESG 智慧的 Clarity AI，正在引領這項轉型。該行業的成長受到日益增加的監管壓力和消費者對符合全球永續發展目標的透明、環保金融服務的需求不斷增長的推動。

8. 先買後付（BNPL）
預計「先買後付」 （BNPL）將在 2025 年實現成長，隨著信用評估模型的增強和監管監督的加強，該行業將得到重塑。開放銀行整合將實現更準確的負擔能力檢查，而人工智慧風險評估將有助於降低違約率。產業領導者Klarna繼續透過其購物超級應用程式方法進行創新，而 Block（前身為 Square）則透過 Afterpay 整合鞏固了其地位。在金融監管機構日益嚴格的審查下，該行業的成熟將以整合和更注重負責任的貸款實踐為標誌。

7.監管技術（Regtech）
隨著金融機構尋求更有效的方式來管理監理負擔，監理科技的採用率將在 2025 年增加。人工智慧和機器學習將提高監管報告和風險監控的準確性，儘管實施挑戰仍然存在。自然語言處理將幫助公司更快地處理監管變化，而區塊鏈解決方案則提供更好的 KYC 和交易追蹤功能。 ComplyAdvantage 持續完善其金融犯罪檢測平台，而 Chainalysis 則加強其在加密貨幣合規方面的地位。成長主要受到日益增加的監管壓力以及利潤率收緊的情況下降低合規成本的需要的推動。

6.嵌入式金融
隨著非金融品牌將銀行、支付和貸款融入客戶旅程，嵌入式金融將在 2025 年獲得進一步發展動能。 API-first 基礎設施供應商將使實施更快、更具成本效益，實現從應用程式內保險到情境借貸的一切。銀行即服務提供商 Stripe Treasury 繼續擴大其嵌入式金融產品，而 Plaid 則透過增強數據連接解決方案鞏固其地位。該行業的成長反映了消費者對其所偏好的數位體驗中無縫金融服務的期望的轉變。

5. 中央銀行數位貨幣（CBDC）
2025 年，中央銀行數位貨幣(CBDC) 將在正在進行的試點計畫和試驗中繼續發展。數位人民幣在中國擴大了國內影響力，而歐洲央行則透過受控測試階段推進其數位歐元計畫。支付技術供應商 G+D（Giesecke+Devrient）加強了其在 CBDC 基礎設施開發中的地位，而 R3 的 Corda 平台支援各種中央銀行試驗。由於各國央行都在努力應對金融穩定、隱私以及對傳統銀行體系的影響等問題，因此進展仍然有限。

4. 網路安全與生物辨識技術
隨著金融機構打擊複雜的網路威脅和詐欺行為，網路安全和生物辨識認證將在 2025 年變得越來越重要。多模式生物辨識技術將臉部辨識與行為模式結合，將加強安全訪問，而人工智慧威脅偵測系統可以識別新興的攻擊媒介。 OneSpan 繼續推進其行動安全和身份驗證解決方案，而 BioCatch 則加強其在行為生物識別領域的地位。該行業的緊迫性源於日益增長的網路威脅、更強身份驗證的監管要求以及消費者對無摩擦且安全的銀行體驗的需求。

3. 去中心化金融（DeFi）和開放銀行
隨著傳統金融和去中心化系統找到更多共同點，DeFi和開放銀行將在 2025 年面臨複雜的情況。將出現將 DeFi 的可編程性與開放銀行的監管合規性相結合的混合解決方案，尤其是在藉貸和資產代幣化領域。 Aave 協議繼續透過其機構產品連接傳統金融和去中心化金融，而 TrueLayer 則在各個市場擴展其開放銀行基礎設施。持續的監管審查以及平衡創新與消費者保護和穩定的需求阻礙了成長。

2.機器人流程自動化（RPA）
2025 年，金融服務領域的機器人流程自動化(RPA) 將超越基本任務自動化，人工智慧增強型機器人將處理索賠處理和信用評估中更複雜的決策過程。智慧型文檔處理將提高從非結構化財務文檔中提取和驗證資料的準確性。 UiPath 透過先進的認知自動化功能保持其地位，而 Blue Prism 則加強了其在雲端原生智慧自動化方面的產品。隨著交易量的增加，降低營運成本和提高處理速度的壓力推動了成長。

1.人工智慧（AI）和機器學習（ML）
到 2025 年，人工智慧和機器學習將更深入地融入金融服務，產生人工智慧將增強客戶服務互動，預測分析將改善風險評估和詐欺偵測。大型語言模型將有助於監管合規和投資研究，而電腦視覺則將推動文件處理的發展。 H2O.ai 持續為金融機構提供複雜的 ML 解決方案，而 DataRobot 則加強了其用於風險建模的自動化機器學習平台。對人工智慧的負責任部署以及與新興人工智慧治理框架的監管協調的需求抑制了成長。
"""

# 檢查文本長度，這對分析結果的豐富度很重要
if len(text_to_analyze) < 150:
    print(f"警告：您的文本長度只有 {len(text_to_analyze)} 字。")
    print("為了獲得更豐富的分析結果，我建議您使用至少150字的長文本。")

print(f"\n--- 我的研究文本 (共 {len(text_to_analyze)} 字) ---\n")
print(text_to_analyze)
print("\n" + "="*50 + "\n")

# 執行斷詞、詞性標註與命名實體識別
print("--- 正在執行斷詞、詞性標註與命名實體識別，這可能需要一點時間 ---")
# ckiptagger 的處理單位是句子，所以我需要先將文本分割成句子。
# 這裡我做了一個簡單的句子分割，主要是根據常見的中文標點符號。
# 如果文本有更複雜的結構，可能需要更進階的句子分割方法。
sentences = text_to_analyze.replace('。', '。\n').replace('？', '？\n').replace('！', '！\n').split('\n')
sentences = [s.strip() for s in sentences if s.strip()] # 清理掉可能產生的空行

# 執行斷詞 (Word Segmentation)
print(" - 執行斷詞...")
word_s = ws(sentences)

# 執行詞性標註 (Part-of-Speech Tagging)
print(" - 執行詞性標註...")
word_p = pos(word_s)

# 執行命名實體識別 (Named Entity Recognition)
# 提醒自己：ner 函數接受的輸入是多個句子，每個句子是一串詞語。
print(" - 執行命名實體識別...")
ner_results = ner(word_s, word_p)
print("--- 斷詞、詞性標註與命名實體識別處理完成！---")

# 清晰展示處理結果

print("\n--- 斷詞結果概覽 ---")
# 我想將所有斷詞結果整合成一個列表，這樣方便後續進行詞頻統計。
all_words = []
for i, sentence_words in enumerate(word_s):
    # 這裡只展示每個句子斷詞後的前幾個詞，避免輸出過長。
    print(f"句子 {i+1}: {' / '.join(sentence_words[:10])} ... (更多詞)")
    all_words.extend(sentence_words)

print("\n--- 命名實體識別 (NER) 結果概覽 ---")
# 這裡會印出每個句子中識別到的命名實體。
for i, sentence_ner_result in enumerate(ner_results):
    print(f"句子 {i+1} 的實體：")
    if sentence_ner_result:
        for entity in sentence_ner_result:
            # entity 的格式通常是 (起始索引, 結束索引, 實體類型, 實體文本)
            # 例如：(0, 4, 'ORG', '國泰金控')，這裡我主要關注類型和實體本身。
            print(f"  - 類型: {entity[2]}, 實體: {entity[3]}")
    else:
        print("  (此句沒有識別到命名實體)")
print("\n" + "="*50 + "\n")


--- 我的研究文本 (共 2503 字) ---


2025年金融科技十大預測
探索人工智慧、永續性和不斷變化的消費者需求如何在 2025 年改變金融科技，從財富管理到 CBDC 和自動化銀行業務
展望 2025 年，在人工智慧、永續性議題和不斷變化的消費者偏好的推動下，金融科技格局將繼續以驚人的速度發展。
從人工智慧財富管理到綠色金融科技解決方案，這十大趨勢正在重塑我們與金錢和金融服務的互動方式。
創新涉及多個領域，包括網路安全、監管技術和分散金融，而 RPA 和嵌入式金融等成熟技術正在達到新的複雜程度。
在這十大趨勢中，我們探討了 2025 年定義金融科技產業的關鍵發展，為金融的未來提供了見解。
10.財富科技
隨著人工智慧和個人化重塑財富管理，財富科技產業預計在 2025 年實現顯著成長。增強型人工智慧驅動的投資組合管理和自動化稅收優化將變得更加突出，而嵌入式金融解決方案將把財富服務整合到日常應用程式中。擁有先進機器人諮詢平台的 Betterment 和以全面財富數據分析而聞名的 Addepar 等公司正在引領這一轉變。千禧世代和 Z 世代投資者對數位優先財富管理解決方案的需求不斷增長，以及開放銀行標準的日益普及，推動了該行業的擴張。

9.永續與綠色金融科技
綠色金融科技將在 2025 年徹底改變永續金融，碳足跡追蹤將無縫整合到銀行應用程式和支付系統中。先進的 ESG 分析和基於區塊鏈的碳信用交易平台將能夠更精確地衡量環境影響。 Aspiration 憑藉其註重氣候的銀行服務，以及利用人工智慧提供 ESG 智慧的 Clarity AI，正在引領這項轉型。該行業的成長受到日益增加的監管壓力和消費者對符合全球永續發展目標的透明、環保金融服務的需求不斷增長的推動。

8. 先買後付（BNPL）
預計「先買後付」 （BNPL）將在 2025 年實現成長，隨著信用評估模型的增強和監管監督的加強，該行業將得到重塑。開放銀行整合將實現更準確的負擔能力檢查，而人工智慧風險評估將有助於降低違約率。產業領導者Klarna繼續透過其購物超級應用程式方法進行創新，而 Block（前身為 Square）則透過 Afterpay 整合鞏固了其地位。在金融監管機構日益嚴格的審查下，該行業的成熟將以整合和更注重負責任的貸款實踐為標誌。

7.監管技術（Regtech）
隨著金融機構尋

### **步驟五：分析結果視覺化呈現**

#### 光有處理好的數據是不夠的，為了讓我的研究成果更直觀、更容易理解，我將利用 Plotly Express 繪製兩種圖表：

1.**詞頻分析長條圖 (Word Frequency Bar Chart)**：
這個圖表將展示文本中出現頻率最高的詞彙。透過它，我們可以快速了解文本主要在討論哪些主題或關鍵字。為了讓結果更具意義，我會過濾掉單一字元以及一些常見的「停用詞」（例如「的」、「了」、「是」等），這些詞語雖然頻繁出現，但通常對理解文本核心內容的幫助不大。

2.**命名實體類型分佈圓餅圖 (Named Entity Type Distribution Pie Chart)**：
這個圖表將呈現文本中識別出的命名實體類型（如人名、地名、組織名、時間等）的分佈比例。這有助於我理解文本所關注的實體類型，例如是更側重於組織還是時間點的描述。

In [10]:
# 步驟 5: 斷字分析圖表成果 (詞頻分析) - 使用 Plotly Express

print("--- 步驟 5: 斷字分析圖表成果：詞頻分析 (使用 Plotly Express) ---")
# 計算詞彙頻率
word_counts = Counter(all_words)

# 我需要過濾掉單一字元或一些常見的「停用詞」，讓詞頻分析更聚焦於有意義的內容。
# 這裡我定義了一個常見的中文停用詞列表，你可以根據需要進行擴充。
stopwords = set(['的', '了', '是', '和', '在', '與', '等', '而', '為', '也', '更', '以', '其', '對', '將', '個', '就', '從', '到', '等', '來', '去', '會', '能', '讓', '要', '有', '無', '這', '那', '一', '二', '三', '四', '五', '六', '七', '八', '九', '十', '多', '少', '很', '最', '可', '但', '並', '或', '又', '又', '將會', '可以', '進行', '以及', '成為', '透過', '隨著', '使得', '透過', '為', '將', '由於', '對', '但', '而', '及'])

# 過濾詞彙：長度大於1且不在停用詞列表中的詞語
filtered_word_counts = {word: count for word, count in word_counts.items() if len(word) > 1 and word not in stopwords}

# 我想看看詞頻最高的 N 個詞，這裡我設定為 20 個。
top_n = 20
most_common_words = Counter(filtered_word_counts).most_common(top_n)

if most_common_words:
    # 將結果轉換成 pandas DataFrame，Plotly Express 處理 DataFrame 非常方便。
    words_df = pd.DataFrame(most_common_words, columns=['詞彙', '出現次數'])

    # 使用 plotly.express 繪製長條圖
    fig_bar = px.bar(words_df, x='詞彙', y='出現次數',
                     title=f'文本中詞頻最高的 {top_n} 個詞',
                     labels={'詞彙': '詞彙', '出現次數': '詞彙出現次數'}, # 設定軸標籤，讓圖表更清晰
                     color='出現次數', # 根據出現次數給予顏色深淺，增加視覺層次
                     color_continuous_scale=px.colors.sequential.Plasma) # 設定顏色漸變方案

    # 調整圖表佈局，確保中文字體能正確顯示，並讓標題置中
    fig_bar.update_layout(
        xaxis_title_text='關鍵詞彙', # 讓 x 軸標題更具描述性
        yaxis_title_text='出現頻率', # 讓 y 軸標題更具描述性
        title_x=0.5, # 標題置中
        font_family="Noto Sans CJK TC", # 嘗試指定字體，Colab 環境通常支援
        font_size=12, # 調整字體大小，讓圖表資訊更易讀
        height=500, # 設定圖表高度
        width=800 # 設定圖表寬度
    )
    fig_bar.show()
else:
    print("沒有足夠的詞彙可以進行詞頻分析或繪製圖表。請檢查您的文本內容和過濾條件。")


print("\n--- 命名實體類型分佈圖表 (使用 Plotly Express) ---")
# 統計 NER 識別到的實體類型分佈
ner_types = []
for sentence_ner_result in ner_results:
    for entity in sentence_ner_result:
        ner_types.append(entity[2]) # 實體類型通常在 entity[2] 的位置

ner_type_counts = Counter(ner_types)

if ner_type_counts:
    # 同樣地，將結果轉換為 DataFrame
    ner_df = pd.DataFrame(ner_type_counts.items(), columns=['實體類型', '數量'])

    # 使用 plotly.express 繪製圓餅圖
    fig_pie = px.pie(ner_df, values='數量', names='實體類型',
                     title='文本中命名實體類型分佈',
                     hole=0.3, # 設定圓環圖的洞，讓圖表更美觀
                     color_discrete_sequence=px.colors.sequential.RdBu) # 設定顏色序列

    # 調整圖表佈局，確保中文字體能正確顯示，並讓標題置中
    fig_pie.update_layout(
        title_x=0.5, # 標題置中
        font_family="Noto Sans CJK TC", # 嘗試指定字體
        font_size=12, # 調整字體大小
        height=500, # 設定圖表高度
        width=800 # 設定圖表寬度
    )
    fig_pie.show()
else:
    print("文本中沒有識別出任何命名實體，因此無法繪製分佈圖。")

print("\n--- 程式執行完畢！我的研究分析告一段落。---")

--- 步驟 5: 斷字分析圖表成果：詞頻分析 (使用 Plotly Express) ---



--- 命名實體類型分佈圖表 (使用 Plotly Express) ---



--- 程式執行完畢！我的研究分析告一段落。---


### **圖表分析結果**

#### 從 CKIPtagger 對金融科技文本分析結果，我們可以看到以下 insights：

1.**詞頻分析：文本的核心關鍵詞**

根據「文本中詞頻最高的 20 個詞」長條圖，我們可以清楚地看到這篇文本最常討論的主題。最高的詞彙包括「金融」、「智慧」、「銀行」、「人工」、「監管」、「2025」、「科技」、「服務」、「財富」、「自動化」等。這些詞彙高度集中於「金融」、「科技」、「人工智慧」、「監管」以及一個特定的時間點「2025」，這完全符合了文本標題和內容所圍繞的「2025 金融科技預測」的主題。這張圖有效地提煉出了文本的核心討論焦點，顯示了文本對於未來金融科技發展趨勢（特別是與 AI 和自動化相關）的關注。

2.**命名實體分析：文本涉及的實體類型**

「文本中命名實體類型分佈」圓餅圖提供了文本中所提及的實體類型比例。其中佔比最高的三類是：

ORG (組織/機構)：佔比達 50%。這表明文本在討論金融科技時，大量提及了與金融機構、科技公司、監管機構或其他相關組織有關的內容。這與金融科技產業的特性相符，因為它涉及眾多公司和組織的合作與競爭。
DATE (日期/時間)：佔比 31.8%。這個高比例與文本主題「2025 年金融科技預測」直接相關。文本中頻繁出現年份、預測的時間點等，因此日期實體的佔比較高是預期的結果。
CARDINAL (基數詞/數量)：佔比 6.82%。這類實體可能包括報告中的數字、排名前幾的趨勢序號、交易金額等，顯示文本包含了一些量化的資訊。
其餘實體類型如 ORDINAL (序數詞)、PERSON (人名)、GPE (地緣政治實體) 的佔比較小，這表示文本可能較少直接提及具體的人物或特定的國家/地區，而更側重於產業趨勢和相關組織的討論。


### **總結與反思**

#### 這次的 Colab 作業讓我深入體驗了 CKIPtagger 在中文文本處理上的強大功能。從環境設定、模型下載，到斷詞、詞性標註與命名實體識別，每一步都讓我更了解中文 NLP 的運作原理。特別是透過 Plotly Express 進行視覺化，讓原本冰冷的數據變得生動有趣，詞頻圖幫助我快速掌握文本的核心主題，而命名實體分佈則讓我對文本所關注的特定實體類型有了更全面的認識。